In [1]:
import pandas as pd
import numpy as np
np.random.seed(1234) 
import random

In [2]:
# create candidates
candidates = {
    0: ["Elizabeth II", 0.36], 
    1: ["Genghis Khan", 0.35], 
    2: ["Alexander the Great", 0.13], 
    3: ["Mahatma Gandhi", 0.11], 
    4: ["Augustus Caesar", 0.05]
}

In [23]:
class RankChoice:

    def __init__(self, pop_size, turnout_per, print_on=False):
        self.pop_size = pop_size
        self.per_of_turnout = turnout_per
        self.print_on = print_on
        self.votes_pop = pd.DataFrame(index=range(self.pop_size), columns=["Favorite", "Second Choice", "Eh", "Only Cause I Have To", "No Chance"]).fillna(-1)
        self.votes_cast = pd.DataFrame.empty
        self.vote_count = {}
        self.initial_results = {}
        self.eliminated_candidates = []

    def run(self):
        self.getVotePopulation()
        self.dropTurnout()
        self.getVoteCount()
        self.rankChoice()

    def getProbs(self):
        """Return weighted probabilities of each candidate."""
        return [candidates[0][1], candidates[1][1], candidates[2][1], candidates[3][1], candidates[4][1]]

    def getVotePopulation(self):
        """create vote population."""
        # FOR DEMONSTRATION PURPOSES
        # for i in range(5):
        #     newl = np.random.choice([0, 1, 2, 3, 4], size=5, replace=False, p=[0.96, 0.01, 0.01, 0.01, 0.01])
            # print(newl)
            # [0 4 2 1 3]
            # [0 2 4 1 3]
            # [0 2 4 3 1]
            # [0 3 2 4 1]
            # [0 3 1 4 2]
        vote_list = [0, 1, 2, 3, 4]
        ps = self.getProbs()
        for i in range(self.pop_size):
            newl = np.random.choice(vote_list, size=len(vote_list), replace=False, p=ps)
            self.votes_pop.at[i, "Favorite"] = newl[0]
            self.votes_pop.at[i, "Second Choice"] = newl[1]
            self.votes_pop.at[i, "Eh"] = newl[2]
            self.votes_pop.at[i, "Only Cause I Have To"] = newl[3]
            self.votes_pop.at[i, "No Chance"] = newl[4]

    def getVoteCount(self):
        """Count votes."""
        for i in range(len(candidates)):
            indices = self.votes_cast.loc[ (self.votes_cast["Favorite"].iloc[:] == i) | (self.votes_cast["Favorite"].iloc[:] == float(i))].index.values
            self.vote_count[i] = len(indices)
        self.vote_count = dict(sorted(self.vote_count.items(), key=lambda x: x[1], reverse=True))
        if self.print_on:
            self.printResults()

    def printResults(self):
        """Print out the results of the vote count."""
        print({candidates[key][0]:value for key,value in self.vote_count.items()})

    def dropTurnout(self):
        """Remove percentage of votes as non-turnout voters."""
        no_vote = np.random.choice(range(self.votes_pop.shape[0]), size=int(self.votes_pop.shape[0]*self.per_of_turnout), replace=False)
        self.votes_cast = self.votes_pop.drop(no_vote)

    def rankChoice(self):
        """Return results of the election based on rank choice rules."""
        total_votes = sum(self.vote_count.values())
        half_plus_one = int(total_votes/2) + 1
        top_total_votes = list(self.vote_count.values())[0]
        if top_total_votes > half_plus_one:
            # if person with top votes gets a majority, they win
            print(candidates[list(self.vote_count.keys())[0]][0] + " Wins! : " + str(top_total_votes/total_votes))
            return list(self.vote_count.keys())[0]
        else:
            self.runoff()

    def runoff(self):
        """Candidate with fewest first-preference votes is eliminated."""
        few_cand = self.getFewestCandidate()
        self.updateVotesCast(few_cand)
        self.getVoteCount()
        self.rankChoice()

    def getFewestCandidate(self):
        """Get candidate with fewest first-preference votes."""
        fewest_cand = self.votes_cast["Favorite"].value_counts().index.values[-1]
        self.eliminated_candidates.append(int(fewest_cand))
        return fewest_cand

    def updateVotesCast(self, elim_candidate):
        """Update votes_cast df based on eliminated candidate."""
        for i in range(self.votes_cast.shape[0]):
            self.checkForElimVotes(i, elim_candidate)

    def checkForElimVotes(self, i, elim_candidate):
        """Checks votes for individual row to see if next vote is for an eliminated candidate."""
        v1 = self.votes_cast["Favorite"].iloc[i]
        if int(v1) == int(elim_candidate) or int(v1) in self.eliminated_candidates:
            self.shiftVotes(i)
            self.checkForElimVotes(i, elim_candidate)

    def shiftVotes(self, i):
        """Shift given row of votes by one to the left."""
        self.votes_cast.iloc[i, :] = pd.Series(self.votes_cast.iloc[i,:]).shift(-1)



In [24]:
rc = RankChoice(pop_size=100, print_on=True, turnout_per=0.5)
rc.run()

{'Elizabeth II': 20, 'Genghis Khan': 16, 'Alexander the Great': 7, 'Mahatma Gandhi': 6, 'Augustus Caesar': 1}
{'Elizabeth II': 21, 'Genghis Khan': 16, 'Alexander the Great': 7, 'Mahatma Gandhi': 6, 'Augustus Caesar': 0}
{'Elizabeth II': 25, 'Genghis Khan': 18, 'Alexander the Great': 7, 'Mahatma Gandhi': 0, 'Augustus Caesar': 0}
{'Elizabeth II': 29, 'Genghis Khan': 21, 'Alexander the Great': 0, 'Mahatma Gandhi': 0, 'Augustus Caesar': 0}
Elizabeth II Wins! : 0.58
